# openai_api walkthrough（全功能教学版）

## 目标与先修
- 目标：覆盖 `OpenAI` client 全部功能入口，理解每个功能“做什么、参数怎么用、兼容差异怎么看”。
- 先修：Python 基础、JSON 基础、HTTP 状态码基础。


## 流程总览
1. 先构建全功能教学目录（lesson catalog）和全量参数索引。
2. 运行 safe mode 全量 probe，得到每个功能的结构化状态。
3. 逐功能查看“教学卡片 + probe 结果 + 练习题”。
4. 最后运行 `run_full_demo` 汇总总结，再看 side-effect mode（可选）。


### 环境检查（在线模块）
- 本步做什么：检查 `DEEPSEEK_API_KEY` 是否存在。
- 为什么这样做：本 notebook 需要在线 probe 全功能，缺 key 时必须显式失败。
- 输入：环境变量 `DEEPSEEK_API_KEY`。
- 输出：通过则继续；失败则 `RuntimeError`。
- 观察点：失败信息应明确指向配置问题。


In [ ]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY", "")
if not api_key:
    raise RuntimeError("缺少 DEEPSEEK_API_KEY：openai_api 全功能 walkthrough 不能继续执行。")

print("环境检查通过：已检测到 DEEPSEEK_API_KEY")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 和 `show_section`，统一展示结构化对象。
- 为什么这样做：全功能教学内容很长，统一输出格式更容易定位问题。
- 输入：任意对象。
- 输出：可读 JSON 或分节标题。
- 观察点：后续每个功能都复用同一套展示方法。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_section(title: str):
    print(f"\n===== {title} =====")


## 步骤拆解（逐步）

### Step 1: 构建教学目录与参数索引
- 本步做什么：调用 `build_feature_lesson_catalog` 与 `build_full_parameter_reference`。
- 为什么这样做：后续每个功能展示都依赖统一元数据。
- 输入：无。
- 输出：`lessons`、`full_reference`、`lesson_map`。
- 观察点：`lessons` 数量应与 FEATURE_REGISTRY 一致。


In [ ]:
from openai_api.component import (
    build_feature_lesson_catalog,
    build_full_parameter_reference,
    probe_all_features,
    run_demo,
    run_full_demo,
)

user_text = "请用简短中文说明 LLM API 常见能力与参数作用。"

lessons = build_feature_lesson_catalog()
full_reference = build_full_parameter_reference()
lesson_map = {lesson["feature_id"]: lesson for lesson in lessons}

show_section("目录统计")
print("lesson count:", len(lessons))
print("reference count:", len(full_reference))
print("feature ids:", [lesson["feature_id"] for lesson in lessons])


### Step 2: 执行全功能 safe mode probe
- 本步做什么：调用 `probe_all_features(..., include_side_effect_calls=False)`。
- 为什么这样做：先保证低副作用策略下覆盖全部功能，并拿到结构化状态。
- 输入：`user_text`。
- 输出：`probe_results_safe` 和 `probe_map_safe`。
- 观察点：高副作用功能应返回 `skipped_by_policy`，其余功能返回 `ok` 或明确错误分类。


In [ ]:
probe_results_safe = probe_all_features(user_text, include_side_effect_calls=False)
probe_map_safe = {item["feature_id"]: item for item in probe_results_safe}

show_section("safe mode 状态概览")
print([(item["feature_id"], item["status"]) for item in probe_results_safe])

show_section("safe mode 首条结果")
show_json(probe_results_safe[0])


### Step 3: Responses（`responses`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['responses']`、`probe_map_safe['responses']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Responses / 教学卡片")
show_json(lesson_map["responses"])

show_section("Responses / safe mode probe")
show_json(probe_map_safe.get("responses"))

show_section("Responses / API surface")
print("responses.create")


练习题（理解）：理解题：解释 `responses` 与 `responses` 在职责上的差异，并给出一个必须使用 `responses` 的场景。

练习题（动手）：动手题：基于 `responses` 写一个最小示例，记录 status、耗时和失败分类。


### Step 4: Chat Completions（`chat`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['chat']`、`probe_map_safe['chat']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Chat Completions / 教学卡片")
show_json(lesson_map["chat"])

show_section("Chat Completions / safe mode probe")
show_json(probe_map_safe.get("chat"))

show_section("Chat Completions / API surface")
print("chat.completions.create")


练习题（理解）：理解题：解释 `chat` 与 `responses` 在职责上的差异，并给出一个必须使用 `chat` 的场景。

练习题（动手）：动手题：基于 `chat` 写一个最小示例，记录 status、耗时和失败分类。


### Step 5: Completions（`completions`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['completions']`、`probe_map_safe['completions']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Completions / 教学卡片")
show_json(lesson_map["completions"])

show_section("Completions / safe mode probe")
show_json(probe_map_safe.get("completions"))

show_section("Completions / API surface")
print("completions.create")


练习题（理解）：理解题：解释 `completions` 与 `responses` 在职责上的差异，并给出一个必须使用 `completions` 的场景。

练习题（动手）：动手题：基于 `completions` 写一个最小示例，记录 status、耗时和失败分类。


### Step 6: Embeddings（`embeddings`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['embeddings']`、`probe_map_safe['embeddings']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Embeddings / 教学卡片")
show_json(lesson_map["embeddings"])

show_section("Embeddings / safe mode probe")
show_json(probe_map_safe.get("embeddings"))

show_section("Embeddings / API surface")
print("embeddings.create")


练习题（理解）：理解题：解释 `embeddings` 与 `responses` 在职责上的差异，并给出一个必须使用 `embeddings` 的场景。

练习题（动手）：动手题：基于 `embeddings` 写一个最小示例，记录 status、耗时和失败分类。


### Step 7: Moderations（`moderations`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['moderations']`、`probe_map_safe['moderations']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Moderations / 教学卡片")
show_json(lesson_map["moderations"])

show_section("Moderations / safe mode probe")
show_json(probe_map_safe.get("moderations"))

show_section("Moderations / API surface")
print("moderations.create")


练习题（理解）：理解题：解释 `moderations` 与 `responses` 在职责上的差异，并给出一个必须使用 `moderations` 的场景。

练习题（动手）：动手题：基于 `moderations` 写一个最小示例，记录 status、耗时和失败分类。


### Step 8: Files（`files`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['files']`、`probe_map_safe['files']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Files / 教学卡片")
show_json(lesson_map["files"])

show_section("Files / safe mode probe")
show_json(probe_map_safe.get("files"))

show_section("Files / API surface")
print("files.list")


练习题（理解）：理解题：解释 `files` 与 `responses` 在职责上的差异，并给出一个必须使用 `files` 的场景。

练习题（动手）：动手题：基于 `files` 写一个最小示例，记录 status、耗时和失败分类。


### Step 9: Uploads（`uploads`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['uploads']`、`probe_map_safe['uploads']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Uploads / 教学卡片")
show_json(lesson_map["uploads"])

show_section("Uploads / safe mode probe")
show_json(probe_map_safe.get("uploads"))

show_section("Uploads / API surface")
print("uploads.create")


练习题（理解）：理解题：解释 `uploads` 与 `responses` 在职责上的差异，并给出一个必须使用 `uploads` 的场景。

练习题（动手）：动手题：基于 `uploads` 写一个最小示例，记录 status、耗时和失败分类。


### Step 10: Batches（`batches`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['batches']`、`probe_map_safe['batches']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Batches / 教学卡片")
show_json(lesson_map["batches"])

show_section("Batches / safe mode probe")
show_json(probe_map_safe.get("batches"))

show_section("Batches / API surface")
print("batches.list")


练习题（理解）：理解题：解释 `batches` 与 `responses` 在职责上的差异，并给出一个必须使用 `batches` 的场景。

练习题（动手）：动手题：基于 `batches` 写一个最小示例，记录 status、耗时和失败分类。


### Step 11: Models（`models`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['models']`、`probe_map_safe['models']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Models / 教学卡片")
show_json(lesson_map["models"])

show_section("Models / safe mode probe")
show_json(probe_map_safe.get("models"))

show_section("Models / API surface")
print("models.list")


练习题（理解）：理解题：解释 `models` 与 `responses` 在职责上的差异，并给出一个必须使用 `models` 的场景。

练习题（动手）：动手题：基于 `models` 写一个最小示例，记录 status、耗时和失败分类。


### Step 12: Fine Tuning（`fine_tuning`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['fine_tuning']`、`probe_map_safe['fine_tuning']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Fine Tuning / 教学卡片")
show_json(lesson_map["fine_tuning"])

show_section("Fine Tuning / safe mode probe")
show_json(probe_map_safe.get("fine_tuning"))

show_section("Fine Tuning / API surface")
print("fine_tuning.jobs.list")


练习题（理解）：理解题：解释 `fine_tuning` 与 `responses` 在职责上的差异，并给出一个必须使用 `fine_tuning` 的场景。

练习题（动手）：动手题：基于 `fine_tuning` 写一个最小示例，记录 status、耗时和失败分类。


### Step 13: Vector Stores（`vector_stores`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['vector_stores']`、`probe_map_safe['vector_stores']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Vector Stores / 教学卡片")
show_json(lesson_map["vector_stores"])

show_section("Vector Stores / safe mode probe")
show_json(probe_map_safe.get("vector_stores"))

show_section("Vector Stores / API surface")
print("vector_stores.list")


练习题（理解）：理解题：解释 `vector_stores` 与 `responses` 在职责上的差异，并给出一个必须使用 `vector_stores` 的场景。

练习题（动手）：动手题：基于 `vector_stores` 写一个最小示例，记录 status、耗时和失败分类。


### Step 14: Conversations（`conversations`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['conversations']`、`probe_map_safe['conversations']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Conversations / 教学卡片")
show_json(lesson_map["conversations"])

show_section("Conversations / safe mode probe")
show_json(probe_map_safe.get("conversations"))

show_section("Conversations / API surface")
print("conversations.create")


练习题（理解）：理解题：解释 `conversations` 与 `responses` 在职责上的差异，并给出一个必须使用 `conversations` 的场景。

练习题（动手）：动手题：基于 `conversations` 写一个最小示例，记录 status、耗时和失败分类。


### Step 15: Realtime（`realtime`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['realtime']`、`probe_map_safe['realtime']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Realtime / 教学卡片")
show_json(lesson_map["realtime"])

show_section("Realtime / safe mode probe")
show_json(probe_map_safe.get("realtime"))

show_section("Realtime / API surface")
print("realtime.client_secrets.create")


练习题（理解）：理解题：解释 `realtime` 与 `responses` 在职责上的差异，并给出一个必须使用 `realtime` 的场景。

练习题（动手）：动手题：基于 `realtime` 写一个最小示例，记录 status、耗时和失败分类。


### Step 16: Webhooks（`webhooks`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['webhooks']`、`probe_map_safe['webhooks']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Webhooks / 教学卡片")
show_json(lesson_map["webhooks"])

show_section("Webhooks / safe mode probe")
show_json(probe_map_safe.get("webhooks"))

show_section("Webhooks / API surface")
print("webhooks.verify_signature")


练习题（理解）：理解题：解释 `webhooks` 与 `responses` 在职责上的差异，并给出一个必须使用 `webhooks` 的场景。

练习题（动手）：动手题：基于 `webhooks` 写一个最小示例，记录 status、耗时和失败分类。


### Step 17: Evals（`evals`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['evals']`、`probe_map_safe['evals']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Evals / 教学卡片")
show_json(lesson_map["evals"])

show_section("Evals / safe mode probe")
show_json(probe_map_safe.get("evals"))

show_section("Evals / API surface")
print("evals.list")


练习题（理解）：理解题：解释 `evals` 与 `responses` 在职责上的差异，并给出一个必须使用 `evals` 的场景。

练习题（动手）：动手题：基于 `evals` 写一个最小示例，记录 status、耗时和失败分类。


### Step 18: Containers（`containers`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['containers']`、`probe_map_safe['containers']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Containers / 教学卡片")
show_json(lesson_map["containers"])

show_section("Containers / safe mode probe")
show_json(probe_map_safe.get("containers"))

show_section("Containers / API surface")
print("containers.list")


练习题（理解）：理解题：解释 `containers` 与 `responses` 在职责上的差异，并给出一个必须使用 `containers` 的场景。

练习题（动手）：动手题：基于 `containers` 写一个最小示例，记录 status、耗时和失败分类。


### Step 19: Skills（`skills`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['skills']`、`probe_map_safe['skills']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Skills / 教学卡片")
show_json(lesson_map["skills"])

show_section("Skills / safe mode probe")
show_json(probe_map_safe.get("skills"))

show_section("Skills / API surface")
print("skills.list")


练习题（理解）：理解题：解释 `skills` 与 `responses` 在职责上的差异，并给出一个必须使用 `skills` 的场景。

练习题（动手）：动手题：基于 `skills` 写一个最小示例，记录 status、耗时和失败分类。


### Step 20: Videos（`videos`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['videos']`、`probe_map_safe['videos']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Videos / 教学卡片")
show_json(lesson_map["videos"])

show_section("Videos / safe mode probe")
show_json(probe_map_safe.get("videos"))

show_section("Videos / API surface")
print("videos.list")


练习题（理解）：理解题：解释 `videos` 与 `responses` 在职责上的差异，并给出一个必须使用 `videos` 的场景。

练习题（动手）：动手题：基于 `videos` 写一个最小示例，记录 status、耗时和失败分类。


### Step 21: Images（`images`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['images']`、`probe_map_safe['images']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Images / 教学卡片")
show_json(lesson_map["images"])

show_section("Images / safe mode probe")
show_json(probe_map_safe.get("images"))

show_section("Images / API surface")
print("images.generate")


练习题（理解）：理解题：解释 `images` 与 `responses` 在职责上的差异，并给出一个必须使用 `images` 的场景。

练习题（动手）：动手题：基于 `images` 写一个最小示例，记录 status、耗时和失败分类。


### Step 22: Audio（`audio`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['audio']`、`probe_map_safe['audio']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Audio / 教学卡片")
show_json(lesson_map["audio"])

show_section("Audio / safe mode probe")
show_json(probe_map_safe.get("audio"))

show_section("Audio / API surface")
print("audio.transcriptions.create")


练习题（理解）：理解题：解释 `audio` 与 `responses` 在职责上的差异，并给出一个必须使用 `audio` 的场景。

练习题（动手）：动手题：基于 `audio` 写一个最小示例，记录 status、耗时和失败分类。


### Step 23: Beta（`beta`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['beta']`、`probe_map_safe['beta']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Beta / 教学卡片")
show_json(lesson_map["beta"])

show_section("Beta / safe mode probe")
show_json(probe_map_safe.get("beta"))

show_section("Beta / API surface")
print("beta.assistants.list")


练习题（理解）：理解题：解释 `beta` 与 `responses` 在职责上的差异，并给出一个必须使用 `beta` 的场景。

练习题（动手）：动手题：基于 `beta` 写一个最小示例，记录 status、耗时和失败分类。


### Step 24: Client with_options/copy（`client_with_options`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['client_with_options']`、`probe_map_safe['client_with_options']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Client with_options/copy / 教学卡片")
show_json(lesson_map["client_with_options"])

show_section("Client with_options/copy / safe mode probe")
show_json(probe_map_safe.get("client_with_options"))

show_section("Client with_options/copy / API surface")
print("client.with_options(...).responses.create")


练习题（理解）：理解题：解释 `client_with_options` 与 `responses` 在职责上的差异，并给出一个必须使用 `client_with_options` 的场景。

练习题（动手）：动手题：基于 `client_with_options` 写一个最小示例，记录 status、耗时和失败分类。


### Step 25: Client with_raw_response（`client_with_raw_response`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['client_with_raw_response']`、`probe_map_safe['client_with_raw_response']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Client with_raw_response / 教学卡片")
show_json(lesson_map["client_with_raw_response"])

show_section("Client with_raw_response / safe mode probe")
show_json(probe_map_safe.get("client_with_raw_response"))

show_section("Client with_raw_response / API surface")
print("client.with_raw_response.responses.create")


练习题（理解）：理解题：解释 `client_with_raw_response` 与 `responses` 在职责上的差异，并给出一个必须使用 `client_with_raw_response` 的场景。

练习题（动手）：动手题：基于 `client_with_raw_response` 写一个最小示例，记录 status、耗时和失败分类。


### Step 26: Client with_streaming_response（`client_with_streaming_response`）
- 本步做什么：展示该功能的教学卡片与 safe mode probe 结果。
- 为什么这样做：把“概念层”与“实调层”放在同一视图，便于理解边界。
- 输入：`lesson_map['client_with_streaming_response']`、`probe_map_safe['client_with_streaming_response']`。
- 输出：该功能的参数、示例、练习题与状态结果。
- 观察点：关注 `status/http_status/provider_error_type` 与 `stability/side_effect_level` 的关系。


In [ ]:
show_section("Client with_streaming_response / 教学卡片")
show_json(lesson_map["client_with_streaming_response"])

show_section("Client with_streaming_response / safe mode probe")
show_json(probe_map_safe.get("client_with_streaming_response"))

show_section("Client with_streaming_response / API surface")
print("client.with_streaming_response.responses.create")


练习题（理解）：理解题：解释 `client_with_streaming_response` 与 `responses` 在职责上的差异，并给出一个必须使用 `client_with_streaming_response` 的场景。

练习题（动手）：动手题：基于 `client_with_streaming_response` 写一个最小示例，记录 status、耗时和失败分类。


## 端到端结果

### Step 全量总结：运行 run_full_demo 与 core run_demo 对照
- 本步做什么：分别执行 `run_full_demo`（全功能）与 `run_demo`（核心三端点）。
- 为什么这样做：对照全量教学链路和兼容历史链路。
- 输入：同一 `user_text`。
- 输出：`full_demo_result` 与 `core_demo_result`。
- 观察点：全量结果应包含全部 feature 的状态统计；核心结果应保持原有事件顺序。


In [ ]:
full_demo_result = run_full_demo(user_text, include_side_effect_calls=False)
core_demo_result = run_demo(user_text)

show_section("run_full_demo.final_answer")
print(full_demo_result["final_answer"])

show_section("run_demo.final_answer")
print(core_demo_result["final_answer"])


### Step 可选演示：side-effect mode（默认不执行）
- 本步做什么：提供 `include_side_effect_calls=True` 的可选执行单元。
- 为什么这样做：高副作用功能可能产生真实资源或费用，默认不自动触发。
- 输入：`RUN_SIDE_EFFECT_DEMO` 开关。
- 输出：若开启则返回 side-effect 结果；否则打印跳过说明。
- 观察点：仅在你明确确认后再改为 `True`。


In [ ]:
RUN_SIDE_EFFECT_DEMO = False

if RUN_SIDE_EFFECT_DEMO:
    probe_results_side_effect = probe_all_features(user_text, include_side_effect_calls=True)
    show_section("side-effect mode 状态概览")
    print([(item["feature_id"], item["status"]) for item in probe_results_side_effect])
else:
    print("已按默认策略跳过 side-effect mode。")


## 常见错误
- `RuntimeError: 缺少 DEEPSEEK_API_KEY`：环境未配置，属于运行前置问题。
- `unsupported_by_provider`：兼容层未覆盖该功能，不是教学逻辑失败。
- `auth_error/rate_limited/network_error`：分别对应鉴权、限流、网络问题。
- `skipped_by_policy`：高副作用功能在 safe mode 下按策略跳过。

## 总结
- 本 notebook 覆盖了 OpenAI client 全功能入口与 client 操作能力。
- 学习顺序建议：先看每个功能的“教学卡片”，再看 probe 结果，再做练习题。
- 实战排错先看结构化状态，再定位具体端点和参数。
